In [23]:
import pandas as pd
import numpy as np
import csv
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Linear regression

In [24]:
df = pd.read_csv("data_ml_clean.csv")

In [25]:
# One-hot encoding pour les "Country" (chaîne de texte transformée en colonnes numériques)
data_encoded = pd.get_dummies(df, columns=["Country"], drop_first=True)
data_encoded.to_csv("data_ml_clean_OneHot.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

# Définir les features et la cible
features = ["City_Centre","Bedrooms","Salary","Fitness_Club","McDonalds","Water"] + \
           [col for col in data_encoded.columns if col.startswith("Country_")]
X = data_encoded[features]  # Variables explicatives
y = data_encoded["Price"]  # Variable cible


# Diviser les données en entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Créer et entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire les prix sur les données de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = (abs(y_test - y_pred) / abs(y_test)).mean() * 100

print(f"Performances du modèle régression linéaire:")
print(f"Coefficient de détermination (R²) : {r2}")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Mean Absolute Error (MAE) : {mae}")
print(f"Mean Absolute Percentage Error (MAPE) : {mape:.2f}%")

# Créer et entraîner le modèle de régression polynomial
for degree in [2]:
    poly_transformer = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly_transformer.fit_transform(X_train)
    X_test_poly = poly_transformer.transform(X_test)

    model_poly = LinearRegression()
    model_poly.fit(X_train_poly, y_train)
    y_pred = model_poly.predict(X_test_poly)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = (abs(y_test - y_pred) / abs(y_test)).mean() * 100

    print(f"Performances du modèle polynomial:")
    print(f"Coefficient de détermination (R²) : {r2}")
    print(f"Mean Squared Error (MSE) : {mse}")
    print(f"Mean Absolute Error (MAE) : {mae}")
    print(f"Mean Absolute Percentage Error (MAPE) : {mape:.2f}%")


Performances du modèle régression linéaire:
Coefficient de détermination (R²) : 0.7372911169338275
Mean Squared Error (MSE) : 149490.50334791152
Mean Absolute Error (MAE) : 235.73795928232326
Mean Absolute Percentage Error (MAPE) : 48.54%
Performances du modèle polynomial:
Coefficient de détermination (R²) : 0.792993770364702
Mean Squared Error (MSE) : 117793.75369099854
Mean Absolute Error (MAE) : 184.32395490085608
Mean Absolute Percentage Error (MAPE) : 31.38%


In [ ]:
# 1. Créer un DataFrame pour les résultats
results_df = pd.DataFrame({
    "y_test": y_test,
    "y_pred": y_pred
})

# 2. Visualiser les valeurs réelles vs prédites
fig = px.scatter(
    results_df, 
    x="y_test", 
    y="y_pred", 
    labels={"y_test": "Valeurs réelles", "y_pred": "Valeurs prédites"},
    title="Comparaison entre valeurs réelles et prédites (modèle polynomial)"
)

# Ajouter une ligne "parfaite" (y_test == y_pred)
fig.add_shape(
    type="line", line=dict(dash="dash"), 
    x0=min(y_test), y0=min(y_test), 
    x1=max(y_test), y1=max(y_test)
)
# pio.renderers.default = "browser"
fig.update_layout(showlegend=False)
fig.show()


Opening in existing browser session.


In [27]:
# Calcul des résidus
# Les résidus (y_test - y_pred) doivent idéalement être répartis de manière aléatoire autour de zéro.
residuals = y_test - y_pred
residuals_df = pd.DataFrame({"y_test": y_test, "residuals": residuals})

# Résidus en fonction des valeurs réelles
fig = px.scatter(
    residuals_df, 
    x="y_test", 
    y="residuals",
    labels={"y_test": "Valeurs réelles", "residuals": "Résidus"},
    title="Résidus en fonction des valeurs réelles"
)

# Ajouter une ligne horizontale (erreur nulle)
fig.add_shape(
    type="line", line=dict(dash="dash"),
    x0=min(y_test), x1=max(y_test),
    y0=0, y1=0
)

fig.update_layout(showlegend=False)
fig.show()


Opening in existing browser session.


In [28]:
# # Tracer les données (uniquement pour les modèles avec une seule feature X)
# X_range = np.linspace(min(X_train), max(X_train), 200).reshape(-1, 1)
# X_poly_range = poly_transformer.transform(X_range)
# y_pred_range = model_poly.predict(X_poly_range)

# fig = px.scatter(
#     x=X_train.flatten(), 
#     y=y_train, 
#     labels={"x": "X (Feature)", "y": "y (Valeurs Réelles)"},
#     title="Ajustement du modèle polynomial"
# )

# fig.add_traces(
#     px.line(
#         x=X_range.flatten(), 
#         y=y_pred_range, 
#         labels={"x": "X (Feature)", "y": "y"},
#     ).data
# )

# fig.show()
